In [1]:
import boto3
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import os
import pickle

# this is using login to the API via OAuth which enables private data for your own channel

credentials = None

# token.pickle stores the user's credentials from previously successful logins
if os.path.exists('token.pickle'):
    print('Loading credentials from file...')
    with open('token.pickle', 'rb') as token:
        credentials = pickle.load(token)

if not credentials or not credentials.valid:
    if credentials and credentials.expired and credentials.refresh_token:
        print('Refreshing Access Token...')
        credentials.refresh(Request())
    else:
        print('Fetching new tokens...')
        flow = InstalledAppFlow.from_client_secrets_file(
            'client_secrets.json',
            scopes=[
                'https://www.googleapis.com/auth/youtube.readonly',
                'https://www.googleapis.com/auth/yt-analytics.readonly'
                ]
            )
        
        flow.run_local_server(port=8080, prompt='consent')
        # you receive access and refresh tokens
        # refresh token is to get new acccess tokens
        credentials = flow.credentials

        #save
        with open('token.pickle', 'wb') as f:
            print('Saving credentials for future use...')
            pickle.dump(credentials, f)

# Create YouTube API object
youtubeReporting = build('youtubereporting', 'v1', credentials=credentials)

Loading credentials from file...
Refreshing Access Token...


In [1]:
# uncomment this line to get the secret string you need to copy to AWS Secrets Manager
#credentials.to_json()

In [2]:
# Initialize the DynamoDB client
dynamodb = boto3.client('dynamodb')

# Define the specifications for each table
tables = [
    {
        'TableName': 'channel_basic_a2',
        'KeySchema': [
            {'AttributeName': 'composite_key', 'KeyType': 'HASH'},  # Partition key
            {'AttributeName': 'createTime', 'KeyType': 'RANGE'}  # Sort key
        ],
        'AttributeDefinitions': [
            {'AttributeName': 'composite_key', 'AttributeType': 'S'},
            {'AttributeName': 'createTime', 'AttributeType': 'S'}
        ],
        'BillingMode': 'PAY_PER_REQUEST'  # Use on-demand billing mode
    },
    {
        'TableName': 'channel_combined_a2',
        'KeySchema': [
            {'AttributeName': 'composite_key', 'KeyType': 'HASH'},  # Partition key
            {'AttributeName': 'createTime', 'KeyType': 'RANGE'}  # Sort key
        ],
        'AttributeDefinitions': [
            {'AttributeName': 'composite_key', 'AttributeType': 'S'},
            {'AttributeName': 'createTime', 'AttributeType': 'S'}
        ],
        'BillingMode': 'PAY_PER_REQUEST'  # Use on-demand billing mode
    },
    {
        'TableName': 'channel_demographics_a1',
        'KeySchema': [
            {'AttributeName': 'composite_key', 'KeyType': 'HASH'},  # Partition key
            {'AttributeName': 'createTime', 'KeyType': 'RANGE'}  # Sort key
        ],
        'AttributeDefinitions': [
            {'AttributeName': 'composite_key', 'AttributeType': 'S'},
            {'AttributeName': 'createTime', 'AttributeType': 'S'}
        ],
        'BillingMode': 'PAY_PER_REQUEST'  # Use on-demand billing mode
    },
    {
        'TableName': 'channel_sharing_service_a1',
        'KeySchema': [
            {'AttributeName': 'composite_key', 'KeyType': 'HASH'},  # Partition key
            {'AttributeName': 'createTime', 'KeyType': 'RANGE'}  # Sort key
        ],
        'AttributeDefinitions': [
            {'AttributeName': 'composite_key', 'AttributeType': 'S'},
            {'AttributeName': 'createTime', 'AttributeType': 'S'}
        ],
        'BillingMode': 'PAY_PER_REQUEST'  # Use on-demand billing mode
    },
    {
        'TableName': 'reports',
        'KeySchema': [
            {'AttributeName': 'id', 'KeyType': 'HASH'}  # Partition key
        ],
        'AttributeDefinitions': [
            {'AttributeName': 'id', 'AttributeType': 'S'}
        ],
        'BillingMode': 'PAY_PER_REQUEST'  # Use on-demand billing mode
    },
    {
        'TableName': 'traffic_source_type',
        'KeySchema': [
            {'AttributeName': 'id', 'KeyType': 'HASH'}  # Partition key
        ],
        'AttributeDefinitions': [
            {'AttributeName': 'id', 'AttributeType': 'N'}
        ],
        'BillingMode': 'PAY_PER_REQUEST'  # Use on-demand billing mode
    },
        {
        'TableName': 'playback_location_type',
        'KeySchema': [
            {'AttributeName': 'id', 'KeyType': 'HASH'}  # Partition key
        ],
        'AttributeDefinitions': [
            {'AttributeName': 'id', 'AttributeType': 'N'}
        ],
        'BillingMode': 'PAY_PER_REQUEST'  # Use on-demand billing mode
    },
        {
        'TableName': 'traffic_source_detail',
        'KeySchema': [
            {'AttributeName': 'id', 'KeyType': 'HASH'}  # Partition key
        ],
        'AttributeDefinitions': [
            {'AttributeName': 'id', 'AttributeType': 'N'}
        ],
        'BillingMode': 'PAY_PER_REQUEST'  # Use on-demand billing mode
    },
        {   
        'TableName': 'device_type',
        'KeySchema': [
            {'AttributeName': 'id', 'KeyType': 'HASH'}  # Partition key
        ],
        'AttributeDefinitions': [
            {'AttributeName': 'id', 'AttributeType': 'N'}
        ],
        'BillingMode': 'PAY_PER_REQUEST'  # Use on-demand billing mode
    },
        {   
        'TableName': 'operating_system',
        'KeySchema': [
            {'AttributeName': 'id', 'KeyType': 'HASH'}  # Partition key
        ],
        'AttributeDefinitions': [
            {'AttributeName': 'id', 'AttributeType': 'N'}
        ],
        'BillingMode': 'PAY_PER_REQUEST'  # Use on-demand billing mode
    },
        {   
        'TableName': 'sharing_service',
        'KeySchema': [
            {'AttributeName': 'id', 'KeyType': 'HASH'}  # Partition key
        ],
        'AttributeDefinitions': [
            {'AttributeName': 'id', 'AttributeType': 'N'}
        ],
        'BillingMode': 'PAY_PER_REQUEST'  # Use on-demand billing mode
    },
        {   
        'TableName': 'annotations_type',
        'KeySchema': [
            {'AttributeName': 'id', 'KeyType': 'HASH'}  # Partition key
        ],
        'AttributeDefinitions': [
            {'AttributeName': 'id', 'AttributeType': 'N'}
        ],
        'BillingMode': 'PAY_PER_REQUEST'  # Use on-demand billing mode
    }
]

In [6]:
# Create each DynamoDB table
for table in tables:
    response = dynamodb.create_table(**table)
    print(f"Created table: {response['TableDescription']['TableName']}")

Created table: traffic_source_detail
Created table: device_type
Created table: operating_system
Created table: sharing_service
Created table: annotations_type


Set up mapping tables

In [22]:
def populate_mapping_data(table_name, mappings):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table(table_name)
    
    with table.batch_writer() as batch:
        for mapping in mappings:
            batch.put_item(Item=mapping)
    
    print(f"Mapping data inserted into table '{table_name}' successfully.")

# Example mappings data
example_mappings = [
    {'id': 1, 'name': 'Category 1'},
    {'id': 2, 'name': 'Category 2'},
    {'id': 3, 'name': 'Category 3'},
    # Add more mappings as needed
]

# Usage
#populate_mapping_data('MappingTable', example_mappings)


In [20]:
traffic_source_detail_mappings = [
    {'id': 1, 'name': 'type of advertisement'},
    {'id': 3, 'name': 'YouTube feature'},
    {'id': 4, 'name': 'channel ID of referral'},
    {'id': 5, 'name': 'search term'},
    {'id': 7, 'name': 'video referral'},
    {'id': 8, 'name': 'type of YouTube page referral'},
    {'id': 9, 'name': 'external page referral'},
    {'id': 17, 'name': 'type of notification referral'},
    {'id': 19, 'name': 'video referral'},
    {'id': 20, 'name': 'video referral'},
    {'id': 25, 'name': 'product ID'},
    {'id': 26, 'name': 'hashtag'},
    {'id': 27, 'name': 'video referral'},
    {'id': 28, 'name': 'channel ID via live redirect'},
    {'id': 30, 'name': 'video referral'}
]

device_type_mappings = [
    {'id': 100, 'name': 'Unknown'},
    {'id': 101, 'name': 'Computer'},
    {'id': 102, 'name': 'TV'},
    {'id': 103, 'name': 'Game console'},
    {'id': 104, 'name': 'Mobile phone'},
    {'id': 105, 'name': 'Tablet'}
]

operating_system_mappings = [
    {'id': 1, 'name': 'Other'},
    {'id': 2, 'name': 'Windows'},
    {'id': 3, 'name': 'Windows Mobile'},
    {'id': 4, 'name': 'Android'},
    {'id': 5, 'name': 'iOS'},
    {'id': 6, 'name': 'Symbian'},
    {'id': 7, 'name': 'Blackberry'},
    {'id': 9, 'name': 'Macintosh'},
    {'id': 10, 'name': 'PlayStation'},
    {'id': 11, 'name': 'Bada'},
    {'id': 12, 'name': 'WebOS'},
    {'id': 13, 'name': 'Linux'},
    {'id': 14, 'name': 'Hiptop'},
    {'id': 15, 'name': 'MeeGo'},
    {'id': 16, 'name': 'Wii'},
    {'id': 17, 'name': 'Xbox'},
    {'id': 18, 'name': 'PlayStation Vita'},
    {'id': 19, 'name': 'Smart TV'},
    {'id': 20, 'name': 'Nintendo 3DS'},
    {'id': 21, 'name': 'Chromecast'},
    {'id': 22, 'name': 'Tizen'},
    {'id': 23, 'name': 'Firefox'},
    {'id': 24, 'name': 'RealMedia'},
    {'id': 25, 'name': 'KaiOS'},
    {'id': 26, 'name': 'Roku'},
    {'id': 27, 'name': 'Nintendo Switch'},
    {'id': 28, 'name': 'Apple tvOS'},
    {'id': 29, 'name': 'Fire OS'},
    {'id': 30, 'name': 'ChromeOS'},
    {'id': 31, 'name': 'Vidaa'}
    ]

sharing_service_mappings = [
    {'id': 0, 'name': 'Unknown'},
    {'id': 1, 'name': 'Digg'},
    {'id': 4, 'name': 'reddit'},
    {'id': 5, 'name': 'StumbleUpon'},
    {'id': 6, 'name': 'mixi'},
    {'id': 7, 'name': 'Yahoo! Japan'},
    {'id': 8, 'name': 'goo'},
    {'id': 9, 'name': 'Ameba'},
    {'id': 10, 'name': 'Facebook'},
    {'id': 11, 'name': 'Myspace'},
    {'id': 12, 'name': 'NUjij'},
    {'id': 18, 'name': 'Tuenti'},
    {'id': 20, 'name': 'menéame'},
    {'id': 21, 'name': 'Wykop'},
    {'id': 22, 'name': 'Skyrock'},
    {'id': 25, 'name': 'Fotka'},
    {'id': 28, 'name': 'hi5'},
    {'id': 31, 'name': 'Twitter'},
    {'id': 32, 'name': 'Cyworld'},
    {'id': 34, 'name': 'Blogger'},
    {'id': 36, 'name': 'VKontakte (ВКонтакте)'},
    {'id': 37, 'name': 'Rakuten (楽天市場)'},
    {'id': 38, 'name': 'LiveJournal'},
    {'id': 39, 'name': 'Odnoklassniki (Одноклассники)'},
    {'id': 40, 'name': 'tumblr.'},
    {'id': 42, 'name': 'Linkedin'},
    {'id': 43, 'name': 'Google+'},
    {'id': 44, 'name': 'Weibo'},
    {'id': 45, 'name': 'Pinterest'},
    {'id': 46, 'name': 'Email'},
    {'id': 47, 'name': 'Facebook Messenger'},
    {'id': 49, 'name': 'WhatsApp'},
    {'id': 50, 'name': 'Hangouts'},
    {'id': 51, 'name': 'Gmail'},
    {'id': 52, 'name': 'Kakao (Kakao Talk)'},
    {'id': 53, 'name': 'Other'},
    {'id': 55, 'name': 'Copy to Clipboard'},
    {'id': 59, 'name': 'Embed'},
    {'id': 60, 'name': 'Text message'},
    {'id': 61, 'name': 'Android messaging'},
    {'id': 62, 'name': 'Verizon messages'},
    {'id': 63, 'name': 'HTC text message'},
    {'id': 64, 'name': 'Sony Conversations'},
    {'id': 65, 'name': 'Go SMS'},
    {'id': 66, 'name': 'LGE Email'},
    {'id': 67, 'name': 'Line'},
    {'id': 68, 'name': 'Viber'},
    {'id': 69, 'name': 'Kik'},
    {'id': 70, 'name': 'Skype'},
    {'id': 71, 'name': 'Blackberry Messenger'},
    {'id': 72, 'name': 'WeChat'},
    {'id': 73, 'name': 'KAKAO Story'},
    {'id': 74, 'name': 'Dropbox'},
    {'id': 75, 'name': 'Telegram'},
    {'id': 76, 'name': 'Facebook Pages'},
    {'id': 77, 'name': 'GroupMe'},
    {'id': 78, 'name': 'Android Email'},
    {'id': 79, 'name': 'Motorola Messaging'},
    {'id': 80, 'name': 'Nearby Share'},
    {'id': 81, 'name': 'Naver'},
    {'id': 82, 'name': 'iOS System Activity Dialog'},
    {'id': 83, 'name': 'Google Inbox'},
    {'id': 84, 'name': 'Android Messenger'},
    {'id': 85, 'name': 'YouTube Music'},
    {'id': 86, 'name': 'YouTube Gaming'},
    {'id': 87, 'name': 'YouTube Kids'},
    {'id': 88, 'name': 'YouTube TV'}
 ]

annotations_type_mappings = [
    {'id': 0, 'name': 'Unknown'},
    {'id': 1, 'name': 'Note'},
    {'id': 3, 'name': 'Spotlight'},
    {'id': 4, 'name': 'Title'},
    {'id': 8, 'name': 'Speech bubble'},
    {'id': 9, 'name': 'Label'},
    {'id': 10, 'name': 'Branding watermark'},
    {'id': 11, 'name': 'Featured video'},
    {'id': 12, 'name': 'Featured playlist'},
    {'id': 30, 'name': 'Call-to-Action'}
 ]

In [23]:
populate_mapping_data('annotations_type', annotations_type_mappings)

Mapping data inserted into table 'annotations_type' successfully.


In [24]:
populate_mapping_data('sharing_service', sharing_service_mappings)

Mapping data inserted into table 'sharing_service' successfully.


In [25]:
populate_mapping_data('traffic_source_detail', traffic_source_detail_mappings)

Mapping data inserted into table 'traffic_source_detail' successfully.


In [26]:
populate_mapping_data('operating_system', operating_system_mappings)

Mapping data inserted into table 'operating_system' successfully.


In [27]:
populate_mapping_data('device_type', device_type_mappings)

Mapping data inserted into table 'device_type' successfully.
